In [2]:
import os, sys 
import pandas as pd
pd.options.display.max_columns=500
pd.options.display.max_colwidth=1000
import matplotlib.pyplot as plt
from tqdm import tqdm 
tqdm.pandas()
import re 

from collections import Counter, defaultdict, OrderedDict

import json 
# import seaborn as sns
# from copy import deepcopy
# from wn.similarity import path

# from tabulate import tabulate, simple_separated_format
# from inflection import singularize, pluralize
# from df_to_latex import DataFrame2Latex
# from collections import Counter, defaultdict
# from inflection import singularize, pluralize


In [2]:
import random
top_k_anchors=5

d = {'lapins': 1.0, 'furs': 0.5, 'pelts': 0.5, 'bunnies': 0.5, 'cottontails': 0.5, 
     'leporide': 0.5, 'leporid': 0.5, 'lambskin': 0.5, 'astrakhan': 0.5, 'bearskin': 0.5}

# Sort the dictionary by values in descending order
sorted_d = sorted(d.items(), key=lambda x: x[1], reverse=True)

# Initialize variables to keep track of the top words and their scores
top_words = []
top_scores = []

# Iterate over the sorted dictionary and add words to the top_words list
# If a word has the same score as the previous word, append it to the top_words list
# If a word has a different score than the previous word, choose up to 5 random words from the top_words list
if len(sorted_d) <= top_k_anchors:
    return d 
else:     
    for i, (word, score) in enumerate(sorted_d):
        if i == 0:
            top_words.append(word)
            top_scores.append(score)
        else:
            if score == top_scores[-1]:
                top_words.append(word)
            else:
                if len(top_words) < 5:
                    top_words.append(word)
                    top_scores.append(score)
                else:
                    rand_index = random.randint(0, len(top_words)-1)
                    if score > top_scores[rand_index]:
                        top_words[rand_index] = word
                        top_scores[rand_index] = score

print(top_words)


['lapins', 'furs', 'pelts', 'bunnies', 'cottontails', 'leporide', 'leporid', 'lambskin', 'astrakhan', 'bearskin']


In [17]:
import random
top_k_anchors = 5

def sample_top_k_anchors_from_a_dict(d, top_k_anchors):
    '''
    return the top k anchors ranked by their scores and ties are broken by random sample 
    '''

    # sort the dictionary by value in descending order
    sorted_dict = sorted(d.items(), key=lambda x: x[1], reverse=True)

    # group the words by their score
    score_groups = {}
    for word, score in sorted_dict:
        if score not in score_groups:
            score_groups[score] = [word]
        else:
            score_groups[score].append(word)

    # randomly select up to 5 words from each score group
    top_words = []
    for score, words in score_groups.items():
        random.shuffle(words)
        top_words.extend(words[:min(top_k_anchors - len(top_words), len(words))])
        if len(top_words)> top_k_anchors: break 
print(top_words)
d = {'lapins': 1.0, 'furs': 0.5, 'pelts': 0.5, 'bunnies': 0.3, 'cottontails': 0.3, 'leporide': 0.3, 'leporid': 0.3, 'lambskin': 0.3, 'astrakhan': 0.25, 'bearskin': 0.2}


['lapins', 'pelts', 'furs', 'leporid', 'bunnies']


In [49]:
import random
import heapq
import itertools 


def sample_top_k_anchors_from_a_dict(d, top_k_anchors):
    # get the top k items from the dictionary based on their scores
    # top_items = heapq.nlargest(top_k_anchors, d.items(), key=lambda x: x[1])
    # print(top_items) 
    # randomly select up to 5 words from each score group
    
    if len(d.keys()) <= top_k_anchors:
        return list(d.keys())
    
    top_words = []
    for score, words in itertools.groupby(d.items(), lambda x: x[1]):
        words = [word for word, score in words]
        random.shuffle(words)
        top_words.extend(words[:min(top_k_anchors - len(top_words), len(words))])
        if len(top_words) == top_k_anchors: 
            return top_words

    

top_k_anchors = 5

d = {'lapins': 1.0, 'furs': 0.5, 'pelts': 0.5, 'bunnies': 0.3, 'cottontails': 0.3, 'leporide': 0.3, 'leporid': 0.3, 'lambskin': 0.3, 'astrakhan': 0.25, 'bearskin': 0.2}
top_words = sample_top_k_anchors_from_a_dict(d, top_k_anchors)
print(top_words)

d = {'quicklime': 1.0, 'calx': 1.0, 'hydroxide': 0.5, 'oxides': 0.5, 'potash': 0.3333, 'titanias': 0.3333, 'pentoxides': 0.3333, 'dioxides': 0.3333, 'monoxides': 0.3333, 'peroxides': 0.3333, 'minium': 0.3333, 'silica': 0.3333, 'superoxides': 0.3333, 'tetroxide': 0.3333, 'trioxide': 0.3333, 'zirconias': 0.3}
top_words = sample_top_k_anchors_from_a_dict(d, top_k_anchors)
print(top_words)
     

['lapins', 'pelts', 'furs', 'leporid', 'bunnies']
['calx', 'quicklime', 'oxides', 'hydroxide', 'superoxides']


In [ ]:

path = '../log/word_to_cohyponyms_score_pl.json'
data = json.load(open(path, 'r'))
df = pd.DataFrame(data)

In [ ]:
df.head()

In [10]:
from nltk.corpus import wordnet as wn
import pandas as pd
import os, sys 
import wn
import json
from wn.similarity import path
ewn = wn.Wordnet('ewn:2020')


def read_cohyponyms(path = '../log/word_to_cohyponyms.txt'):
    if os.path.exists(path):
        print(f"reading cohyponyms: {path}")
        df = pd.read_csv(path)
        df['cohyponyms'] = df['cohyponyms'].apply(lambda x: eval(x))
        word_to_cohyponym = dict(zip(df['word'].to_list(), df['cohyponyms'].to_list()))
        return word_to_cohyponym
    print(f"{path} not found")

def read_cohyponyms_with_score(path = '../log/word_to_cohyponyms_score.json'):
    if os.path.exists(path):
        print(f"reading cohyponyms: {path}")
        word_to_cohyponym_score = json.load(open(path, 'r'))
        word_to_cohyponym = {k: list(v.keys()) for k,v in word_to_cohyponym_score.items()}
        return word_to_cohyponym
    print(f"{path} not found")

In [12]:
word_to_wn_cohyponyms = read_cohyponyms_with_score(path = '../log/word_to_cohyponyms_score_pl.json') #word is singular or plural? 


reading cohyponyms: ../log/word_to_cohyponyms_score_pl.json


In [20]:
# word_to_wn_cohyponyms['piano']
word_to_cohyponym_score = json.load(open('../log/word_to_cohyponyms_score_pl.json', 'r'))
word_to_cohyponym_score['piano']
# goal 

{'pianoforte': 1.0,
 'grand': 0.5,
 'pianola': 0.5,
 'upright': 0.5,
 'spinets': 0.3333,
 'gong': 0.3333,
 'clavichord': 0.3333,
 'klavier': 0.3333,
 'clavier': 0.3333,
 'dulcimers': 0.3333,
 'cithers': 0.3333,
 'accordions': 0.3333,
 'synthesiser': 0.3333,
 'synthesizers': 0.3333,
 'bones': 0.3333,
 'clappers': 0.3333,
 'castanets': 0.3333,
 'chimes': 0.3333,
 'cymbal': 0.3333,
 'tympan': 0.3333,
 'drum': 0.3333,
 'membranophones': 0.3333,
 'glockenspiel': 0.3333,
 'kettledrum': 0.3333,
 'timpani': 0.3333,
 'tympani': 0.3333,
 'lagerphone': 0.3333,
 'maraca': 0.3333,
 'xylophone': 0.3333,
 'marimba': 0.3333,
 'vibraharps': 0.3333,
 'vibes': 0.3333,
 'vibraphone': 0.3333,
 'banjos': 0.3333,
 'strings': 0.3333,
 'chordophones': 0.3333,
 'guitars': 0.3333,
 'kotos': 0.3333,
 'psalteries': 0.3333,
 'shamisens': 0.3333,
 'samisen': 0.3333,
 'sitars': 0.3333,
 'zithern': 0.3333,
 'zither': 0.3333,
 'celesta': 0.3333,
 'harpsichord': 0.25,
 'cembalos': 0.25,
 'contrabasses': 0.25,
 'harps': 

In [19]:
path = '../../log/bert-large-uncased/ALL/SWOWStrength/df_all_use_global_dap_True_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_anchor_source_WordNet_wns_False_wnf_False_swow_score_source_None.HYPERNYMSUITE.csv'
df1 = pd.read_csv(path)
df1.query("sub_label_sg == 'piano'")[['subj_anchors_wordnet', 'subj_anchors_wordnet_sg']]

# df1['subj_anchors_wordnet_num'] = df1['subj_anchors_wordnet'].apply(lambda x: len(eval(x)) ) 
# # df1[['sub_label', 'subj_anchors_wordnet']]

# df1['subj_anchors_wordnet_num'].value_counts()

,subj_anchors_wordnet,subj_anchors_wordnet_sg
7,"['pianoforte', 'pianola', 'spinet', 'gong', 'clavichord', 'klavier', 'dulcimer', 'cither', 'accordion', 'bones', 'clappers', 'castanets', 'chime', 'cymbal', 'tympan', 'drum', 'membranophone', 'glockenspiel', 'kettledrum', 'timpani', 'tympani', 'lagerphone', 'maraca', 'xylophone', 'marimba', 'vibraharp', 'vibes', 'vibraphone', 'banjo', 'chordophone', 'guitar', 'koto', 'psaltery', 'shamisen', 'samisen', 'sitar', 'zithern', 'zither', 'celesta', 'harpsichord', 'cembalo', 'contrabass', 'harp', 'carillon', 'handbell', 'zill', 'bongo', 'tabour', 'tabor', 'tambourine', 'timbrel', 'cello', 'violoncello', 'viol', 'violin', 'fiddle', 'balalaika', 'mandolin', 'citole', 'gittern', 'cithern', 'cittern', 'uke', 'ukulele', 'calliope', 'virginal', 'tuba', 'sousaphone', 'gamba', 'stradavarius', 'strad', 'lyre', 'mandola', 'bombard', 'theremin', 'kazoo', 'ocarina', 'pipework', 'woodwind', 'bombardon', 'clavier', 'euphonium', 'helicon', 'horn', 'bell', 'bugle', 'clarion', 'trumpet', 'cornet', 'fluegel...","['pianoforte', 'pianola', 'spinet', 'gong', 'clavichord']"


In [14]:
path1 = "../../log/bert-large-uncased/ALL/swow_rw/df_all_use_global_dap_True_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_anchor_source_WordNet_wns_False_wnf_False_swow_score_source_None.ALLSWOW.csv"
df1 = pd.read_csv(path1)
query_words = ['bag', 'paino', 'banana', 'van', 'shirt'] #, 'iron']
df1.query(f'sub_label_sg in {query_words}')[['sub_label_sg', 'obj_label', 'subj_anchors_wordnet_sg']] #.sample(30)


,sub_label_sg,obj_label,subj_anchors_wordnet_sg
3,bag,['luggage'],"['mailbag', 'pannier', 'backpack', 'haversack', 'knapsack']"
16,banana,['food'],"['plantain', 'abaca', 'herb', 'bladderpod', 'wintergreen']"
17,van,['artifact'],"['vanguard', 'division', 'troop', 'corps', 'regiment']"
35,shirt,['artifact'],"['camis', 'daishiki', 'dashiki', 'jersey', 'kurta']"
71,bag,['object'],"['mailbag', 'pannier', 'backpack', 'haversack', 'knapsack']"
122,shirt,['clothes'],"['camis', 'daishiki', 'dashiki', 'jersey', 'kurta']"
135,shirt,['apparel'],"['camis', 'daishiki', 'dashiki', 'jersey', 'kurta']"
136,shirt,['commodity'],"['camis', 'daishiki', 'dashiki', 'jersey', 'kurta']"
183,shirt,['object'],"['camis', 'daishiki', 'dashiki', 'jersey', 'kurta']"
257,shirt,['good'],"['camis', 'daishiki', 'dashiki', 'jersey', 'kurta']"


In [17]:
path3 = "../../log/bert-large-uncased/ALL/SWOWStrength/exp_data_results_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_False_cpt_False_anchor_source_SWOW_swow_score_source_None.HYPERNYMSUITE.csv"
df3 = pd.read_csv(path3)
query_words = ['bag', 'paino', 'banana', 'van', 'shirt'] #, 'iron']
df3.query(f'sub_label_sg in {query_words}')[['sub_label_sg', 'obj_label', 'subj_anchors_swow_sg']] #.sample(30)



,sub_label_sg,obj_label,subj_anchors_swow_sg
3,bag,['luggage'],"['tote', 'satchel', 'sack', 'purse', 'plastic']"
16,banana,['food'],"['yellow', 'peel', 'split', 'ripe', 'unripe']"
17,van,['artifact'],"['car', 'moving', 'truck', 'vehicle', 'automobile']"
36,shirt,['artifact'],"['blouse', 'sleeve', 'button', 'polos', 'plaid']"
73,bag,['object'],"['tote', 'satchel', 'sack', 'purse', 'plastic']"
127,shirt,['clothes'],"['blouse', 'sleeve', 'button', 'polos', 'plaid']"
140,shirt,['apparel'],"['blouse', 'sleeve', 'button', 'polos', 'plaid']"
141,shirt,['commodity'],"['blouse', 'sleeve', 'button', 'polos', 'plaid']"
188,shirt,['object'],"['blouse', 'sleeve', 'button', 'polos', 'plaid']"
264,shirt,['good'],"['blouse', 'sleeve', 'button', 'polos', 'plaid']"


In [6]:
path2 = "../../log/bert-large-uncased/ALL/swow_rw/exp_data_results_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_False_cpt_False_anchor_source_LM_swow_score_source_AddSWOWStrength.HYPERNYMSUITE.csv"
df2 = pd.read_csv(path2)
df2[['sub_label', 'obj_label', 'subj_anchors_sg']]



,sub_label,obj_label,subj_anchors_sg
0,catfish,['fish'],"['bass', 'carp', 'shrimp', 'perch', 'alligator']"
1,lemon,['produce'],"['lime', 'orange', 'yellow', 'apple', 'yes']"
2,turtles,['beast'],"['snake', 'bird', 'fish', 'lizard', 'frog']"
3,bags,['luggage'],"['box', 'paper', 'yes', 'shoe', 'case']"
4,rabbits,['creature'],"['fox', 'deer', 'squirrels', 'mouse', 'rodent']"
...,...,...,...
2089,worlds,['everything'],"['human', 'dimension', 'universe', 'people', 'planet']"
2090,worlds,['experience'],"['human', 'dimension', 'universe', 'people', 'planet']"
2091,worlds,['place'],"['human', 'dimension', 'universe', 'people', 'planet']"
2092,writers,['person'],"['artist', 'author', 'poet', 'journalist', 'editor']"


In [37]:
import os, sys 
import pandas as pd
pd.options.display.max_columns=500
pd.options.display.max_colwidth=1000
import matplotlib.pyplot as plt
from tqdm import tqdm 
tqdm.pandas()
import re 

from collections import Counter, defaultdict, OrderedDict
import seaborn as sns
from copy import deepcopy
from wn.similarity import path

from tabulate import tabulate, simple_separated_format
from inflection import singularize, pluralize
from df_to_latex import DataFrame2Latex
from collections import Counter, defaultdict
from inflection import singularize, pluralize

from utils_path import dataset_to_respath
# # Using WN, WN.taxonomy to retrieve path distance 
# - Pointers: 
#     - https://wn.readthedocs.io/en/latest/setup.html
#     - https://wn.readthedocs.io/en/latest/api/wn.html
# 
# - Installation: 
# ```
# !pip install wn
# !pip install wn[web]
# wn.download('ewn:2020')
# ```

# get the co-hyponyms from WordNet (Shick and Schutze, 2020)
# - get the hypernyms, maxinum d(x,y) is 2
# - get the top 2 most frequent senses of each hypernym 
# - get hyponyms of each hypernyms, maxinum distance d(y,z) is 4 
# - constrain the depeth of hypernyms to be 6

# In[201]:


import re 
import wn, wn.taxonomy
ewn = wn.Wordnet('ewn:2020')

def test_min_depth():
    for word in ['concept', 'thought', 'living thing', 'whole', 'psychological feature', 'unit', 'artifact', 'abstraction', 'object','physical entity', 'entity']:
        synset = wn.synsets(word, pos='n')[0]
        min_depth = wn.taxonomy.min_depth(synset, simulate_root=False)
        print(word, min_depth)


def get_inherited_hypernyms(word, k_synset, max_path_hyper, min_taxo_depth=6, print_flag=False):
    '''
    k_synset: the most frequent k_synset of word
    max_path_hyper: up to k level of hypernyms, e.g., 2 level higher than word 
    min_taxo_depth=6: concept, exluded hypernyms: unit, object, artifact, entity
    '''
    
    hyper_synsets = []
    for i, synset in enumerate(wn.synsets(word, pos='n')[:k_synset]): #top K senses of word 
        #print(f"{word} synset {i+1}")
        for j, path in enumerate(wn.taxonomy.hypernym_paths(synset)): #retrieve the hyper path for each synset 
            #print(f"path {j}")
            for i, ss in enumerate(path[:max_path_hyper]): # get the hypernyms within max_path_hyper
                ss_min_txo_depth = wn.taxonomy.min_depth(ss, simulate_root=False) 
                if ss_min_txo_depth< min_taxo_depth: 
                    if print_flag: 
                        print(f"ss:{ss} ss_min_txo_depth: {ss_min_txo_depth}")
                    continue  #remove general concepts like "entity", 'physical entity'
                hyper_synsets.append(ss)
                if print_flag: 
                    print(' ' * i, ss, ss.lemmas()[0], ss_min_txo_depth)
                    
    return hyper_synsets

def get_direct_hyonyms(synsets):
    '''
    Return the direct hyponyms of a given list of synsets
    ''' 
    sister_synsets = []
    for synset in synsets: 
        sister_synsets.extend(synset.hyponyms() )
    return sister_synsets


def get_inherited_hyponyms(initial_synsets, max_path_hypo):
    synsets = initial_synsets
    synsets_hyponyms = []
    
    while max_path_hypo>0:
        synsets = get_direct_hyonyms(synsets)
        synsets_hyponyms.extend(synsets)
        max_path_hypo -=1
        #print(dist)
        #print(synsets)
        #print("-"*80)
    #print(Counter(synsets_all).most_common())
    return synsets_hyponyms


def filter_cohyponyms(word, synsets_cohyponyms, top_k=50):
    cohyponyms = []
    path_scores = defaultdict()
    for synset in synsets_cohyponyms:
        score = path(synset, synset2)
        for lemma in synset.lemmas():
            if lemma == word: continue 
            if len(lemma.split(" ")) >1 or len(lemma.split("-")) >1: continue 
            cohyponyms.append(lemma.lower())
    cohyponyms = Counter(cohyponyms)
    return cohyponyms.most_common(top_k)


    
def get_cohyponyms(word, top_k_cohyonyms=50, top_k_word_synset=2, max_path_hyper=2, max_path_hypo =4, print_flag=False):
    hyper_synsets = get_inherited_hypernyms(word, k_synset=top_k_word_synset, max_path_hyper = max_path_hyper)
    if print_flag:
        for synset in hyper_synsets:
            print("hyper_synsets", synset, synset.lemmas())

    synsets_cohyponyms = get_inherited_hyponyms(hyper_synsets, max_path_hypo= max_path_hypo)
    if print_flag: 
        print("inherited_hyponyms:", synsets_cohyponyms)
        
    concept_cohyponyms = filter_cohyponyms(word, synsets_cohyponyms, top_k=top_k_cohyonyms)
    if print_flag:
        print("concept_cohyponyms: ", concept_cohyponyms)
    return list(dict(concept_cohyponyms).keys())
   

def test_get_cohyponyms(word, test_cohyponyms):
    '''
    word = 'corn'
    test_cohyponyms = ['bean', 'potato', 'barley', 'wheat', 'pea'] 
    word = 'train'
    test_cohyponyms = ['bus', 'plane', 'car', 'tram', 'truck']
    test_get_cohyponyms(word,test_cohyponyms )
    '''
    top_k_cohyonyms = None #200 
    top_k_word_synset = 2
    max_path_hyper = 2
    max_path_hypo = 4

    concept_cohyponyms  = get_cohyponyms(word, top_k_cohyonyms=top_k_cohyonyms, 
                                         top_k_word_synset=top_k_word_synset, 
                                         max_path_hyper=max_path_hyper, max_path_hypo = max_path_hypo)
    
    for query in test_cohyponyms:
        if query in concept_cohyponyms:
            print(query, 'yes')
        else:
            print(query, 'no')
    print(len(concept_cohyponyms), concept_cohyponyms)



# # Evaluation 

# In[225]:


def merge_predictions_in_concept_level(words, uniform_funcion=None, top_k=None ):
    '''
    uniform_function: either signualarize or pluralize 
    '''
    words_uniformed = [uniform_funcion(word) for word in words] if uniform_funcion !=None else words
    concepts = list(OrderedDict.fromkeys(words_uniformed))
    return concepts[:top_k] if top_k is not None else concepts

def concept_evaluation(label, pred):
    '''
    
    label: a list with the singualr and plural labels (e.g., ['tool', 'tools'])
    pred: the top K prediction list 

    return:
        1 if label share with pred else 0  
    '''
    if not isinstance(label, list):
        label = eval(label)
        
    if not isinstance(pred, list):
        pred = eval(pred)

    shared = set(label).intersection(set(pred))
    return 1 if len(shared)>0 else 0 
    # return len(shared)/len(pred)
    

def get_precision_at_k_concept(df, relation, pred_cols, label_col, k_list, pred_col_suffix='obj_mask_'):
    '''
    evalaute model predictions in concept level, ignoring the morphology affects (singular, plural)
    '''

    p_at_x = [] #defaultdict() 
    for pred_col in pred_cols: 
        suffix = pred_col.replace(pred_col_suffix, "")
        prec_cur = defaultdict()
        prec_cur['mask_type'] = suffix
        for k in k_list: 
            df[f'p{k}_{suffix}'] = df[[label_col, pred_col]].apply(lambda x: concept_evaluation(x[0], eval(x[1])[:k] if isinstance(x[1], str) else x[1][:k]), axis=1 )
            prec_cur[f'p@{k}'] = round(df[f'p{k}_{suffix}'].mean() , 3)*100

        p_at_x.append(prec_cur)  
        

    # aggregate the average precision across k 
    df_res = pd.DataFrame(p_at_x) #, columns=['mask_type', 'mAP'])
    df_res['relation'] = [relation]*len(df_res)
    return df_res

def get_highest_mrr_among_labels(label, pred):
    '''
    return the highest rank among the multiple labels. This is applicable to single labels as well, if we the single label is put in a list

    pred: a list of words (candidates)
    label: the true labels, which is a list (different forms of a word, e.g., singular or plurs, like animal and animals)
    '''
    mrr = 0 
    if pred is None: return mrr 

    rank_list = [ pred.index(item) + 1 for item in label if item in pred] 
    if len(rank_list)>0:
        mrr = 1/min(rank_list)

    return mrr 


def get_mrr(df, relation, pred_cols, label_col, pred_col_suffix):
    '''
    mrr is calculated based on the top_k rank, all elements in obj_col are used
    '''

    mrr = [] 
    for i, pred_col in enumerate(pred_cols):
        cur_mrr = defaultdict()
        suffix = pred_col.replace(pred_col_suffix, "")

        df[f'mrr_{suffix}'] = df[[label_col, pred_col]].apply(lambda x: get_highest_mrr_among_labels(x[0], x[1]), axis=1 ) 
        
        cur_mrr['mask_type'] = suffix
        cur_mrr[f"mrr"] = round(df[f'mrr_{suffix}'].mean(), 3)*100
        mrr.append(cur_mrr)

    mrr_df =  pd.DataFrame(data = mrr) #, columns=['mask_type', 'mrr'])
    # mrr_df['mask_type']= mrr_df['mask_type'].apply(lambda x: x.replace(""))
    mrr_df['relation'] = relation
    return mrr_df 


# In[202]:


def get_dataset_to_respath(dataset_to_respath, print_flag=False):
    # remote path 
#     dataset_to_respath = {'hypernymsuite-BLESS': 'log/bert-large-uncased/hypernymsuite/BLESS/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_10_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_True_cpt_False.HYPERNYMSUITE.csv', 'lm_diagnostic_extended-singular': 'log/bert-large-uncased/lm_diagnostic_extended/singular/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_10_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_True_cpt_False.LM_DIAGNOSTIC_EXTENDED.csv', 'clsb-singular': 'log/bert-large-uncased/clsb/singular/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_10_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_True_cpt_False.CLSB.csv', 'hypernymsuite-LEDS': 'log/bert-large-uncased/hypernymsuite/LEDS/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_10_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_True_cpt_False.HYPERNYMSUITE.csv', 'hypernymsuite-EVAL': 'log/bert-large-uncased/hypernymsuite/EVAL/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_10_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_True_cpt_False.HYPERNYMSUITE.csv', 'hypernymsuite-SHWARTZ': 'log/bert-large-uncased/hypernymsuite/SHWARTZ/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_10_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_True_cpt_False.HYPERNYMSUITE.csv'}

    source_dir = 'spartan:~/cogsci/DAP/'
    target_dir = '../../'
    dataset_to_localpath = defaultdict()
    dataset_rename = {
        'hypernymsuite-BLESS': 'BLESS', 'lm_diagnostic_extended-singular': 'DIAG', 'clsb-singular':'CLSB', 'hypernymsuite-LEDS': 'LEDS', 'hypernymsuite-EVAL': 'EVAL', 'hypernymsuite-SHWARTZ': 
        "SHWARTZ"
    }
    for dataset, path in dataset_to_respath.items():
        path = path.replace(".tsv", ".csv")
        source_path = source_dir + path 
        dataset_l1 = dataset.split("-")[0]
        dataset_l2 = dataset.split("-")[1] 
        target_path = target_dir + path
        scp_string = f"!scp {source_path} {target_path}"
        if print_flag:
            print(scp_string)
            print()
#         print(target_path)
        dataset_to_localpath[dataset_rename[dataset]] = target_path 
#     print(dataset_to_localpath)
    return dataset_to_localpath
dataset_to_localpath = get_dataset_to_respath(dataset_to_respath)


from nltk.corpus import wordnet 
wn_lemmas = set(wordnet.all_lemma_names())
def check_word_in_wordnet(word, wn_lemmas):
    '''
    1 if word in wordnet else 0
    '''
    return 1 if word in wn_lemmas else 0 


def get_all_vocab(dataset_to_localpath):
    dataset_to_df = defaultdict()
    vocab_sub = set()
    for dataset, path in dataset_to_localpath.items(): 
        if debug:
           if dataset!='DIAG': continue 
        print("dataset", dataset)
        df = pd.read_csv(path)
        vocab_sub.update(df['sub_label_sg'].to_list())
        print(len(vocab_sub))
    return list(vocab_sub)




## Config for getting co-hyponyms from WordNet
top_k_cohyonyms = None #200 
top_k_word_synset = 2
max_path_hyper = 2
max_path_hypo = 4

#config for evaluation 
pred_col_suffix=''
label_col = 'sub_sister_new'
pred_cols = ['subj_anchors_all_sg']
relation='co-hyponyms'
debug= False #True #eval(sys.argv[1])
print("debug", debug)

# vocab_sub = get_all_vocab(dataset_to_localpath)
word_to_cohyponyms = defaultdict(list)
# for i, word in enumerate(vocab_sub):
#     if i%10==0: print(i)
#     if not check_word_in_wordnet(word, wn_lemmas):
#         word_to_cohyponyms[word] = []
#     else:
#         cohyponyms = get_cohyponyms(word, top_k_cohyonyms=top_k_cohyonyms, 
#                                  top_k_word_synset=top_k_word_synset, 
#                                  max_path_hyper=max_path_hyper, 
#                                  max_path_hypo = max_path_hypo)
#         word_to_cohyponyms[word] = cohyponyms
#     if debug:
#         print(word, cohyponyms)
       
# df = pd.DataFrame(word_to_cohyponyms.items(), columns=['word', 'cohyponyms'])
# output_path = '../log/word_to_cohyponyms.txt'
# df.to_csv(output_path, index=False)
# print(f"save {output_path}")


SyntaxError: invalid syntax (<ipython-input-37-405142216b60>, line 106)

In [26]:
query_words = [ 'mountain', 'mountains' ] #'dog','grain',
for word in query_words: 
    print("-"*40, word, '-'*40)
    cohyponyms = get_cohyponyms(word, top_k_cohyonyms=top_k_cohyonyms, 
                                      top_k_word_synset=top_k_word_synset, 
                                     max_path_hyper=max_path_hyper, 
                                     max_path_hypo = max_path_hypo, 
                                print_flag = True
                               )
    word_to_cohyponyms[word] = cohyponyms
    print(word, cohyponyms)
print(word_to_cohyponyms[word])

---------------------------------------- mountain ----------------------------------------
inherited_hyponyms: []
concept_cohyponyms:  []
mountain []
---------------------------------------- mountains ----------------------------------------
inherited_hyponyms: []
concept_cohyponyms:  []
mountains []
[]


In [13]:
word = 'mountain'
k_synset = 2

get_inherited_hypernyms(word, k_synset, max_path_hyper, min_taxo_depth=6, print_flag=True)

[]

In [39]:
!cd ../script

In [41]:
# !python -u add_path_score_for_cohyponyms.py


import pandas as pd
from collections import defaultdict 
import json 
import wn
from inflection import singularize, pluralize 
import os, sys
from wn.similarity import path
ewn = wn.Wordnet('ewn:2020')


def read_cohyponyms(path = '../log/word_to_cohyponyms.txt'):
    if os.path.exists(path):
        print(f"reading cohyponyms: {path}")
        df = pd.read_csv(path)
        word_to_cohyponym = dict(zip(df['word'], df['cohyponyms']))
        return word_to_cohyponym 
        

    print(f"{path} not found")

    
def get_shortest_path_score(word1, word2):
    synsets1 = ewn.synsets(word1, pos='n')    
    synsets2 = ewn.synsets(word2, pos='n')
    if len(synsets1)==0 or len(synsets2)==0: return 0 
    scores = []
    for synset1 in synsets1:
        for synset2 in synsets2:
            scores.append(path(synset1, synset2  ))
    return max(scores) 

def write_cohyponym_scores(word_to_cohyponyms_score, output_path):
    with open(output_path, 'w') as fout:
        json.dump(word_to_cohyponyms_score, fout, indent=4)
    print(output_path)

def get_path_score_for_cohyponyms(word_to_cohyponyms, debug=False):
    word_to_cohyponyms_score = defaultdict()
    if debug:    
        query_words = ['dog', 'wolf', 'corn', 'car', 'tiger']
    else: 
        query_words = word_to_cohyponyms.keys()
        
    for i, k in enumerate(query_words): 
        k_pl = pluralize(k)
        cur_cohyponyms_to_score = defaultdict()
        if len(word_to_cohyponyms[k])==0: continue 
        for v in eval(word_to_cohyponyms[k]):
            score = get_shortest_path_score(k, v)
            v = pluralize(v)
            cur_cohyponyms_to_score[v] = round(score, 4)
        cur_cohyponyms_to_score = dict(sorted(cur_cohyponyms_to_score.items(), key=lambda x: x[1], reverse=True))
        word_to_cohyponyms_score[k_pl] = cur_cohyponyms_to_score
        if k_pl!=k: 
            word_to_cohyponyms_score[k] = cur_cohyponyms_to_score
        if i%1000==0:
            print(i) 
    return word_to_cohyponyms_score


if __name__=='__main__':
    debug=False 
    # debug=True 
    # start = sys.argv[1]
    # end = sys.argv[2]
    word_to_cohyponyms  = read_cohyponyms(return_hponyms=True)
    # vocab  = vocab[start:end]
    # print(f"processing {start} to {end}")
    word_to_cohyponyms_score = get_path_score_for_cohyponyms(word_to_cohyponyms, debug=debug)
    output_path = '../log/word_to_cohyponyms_score_pl.json'
    write_cohyponym_scores(word_to_cohyponyms_score, output_path)

reading cohyponyms: ../log/word_to_cohyponyms.txt
Traceback (most recent call last):
  File "add_path_score_for_cohyponyms.py", line 65, in <module>
    word_to_cohyponyms  = word_to_cohyponyms[start:end]
TypeError: unhashable type: 'slice'


In [4]:
import pandas as pd 
import os, sys 

def read_cohyponyms(path = '../log/word_to_cohyponyms.txt', chunk_data=False, start=None, end=None):
    if os.path.exists(path):
        print(f"reading cohyponyms: {path}")
        df = pd.read_csv(path)
        print(f"all instances: {len(df.index)}")
        if chunk_data and start!=None and end!=None: 
            df = df.iloc[start:end]
        print("#instances chunk:", len(df.index))
        word_to_cohyponym = dict(zip(df['word'], df['cohyponyms']))
        return word_to_cohyponym 
        

    print(f"{path} not found")
    
read_cohyponyms(path = '../log/word_to_cohyponyms.txt', chunk_data=True, start=100, end=150)

reading cohyponyms: ../log/word_to_cohyponyms.txt
all instances: 12826
#instances chunk: 50


{'montalto': '[]',
 'dreamboat': '[]',
 'gryllus': '[]',
 'kirk': "['cathedral', 'abbey', 'basilica', 'duomo', 'temple', 'tabernacle', 'minster', 'bethel', 'chapel', 'church', 'conventicle', 'meetinghouse', 'masjid', 'musjid', 'mosque', 'shrine', 'synagogue', 'chantry', 'caaba', 'kaaba', 'oracle', 'stupa', 'tope', 'pagoda', 'pantheon', 'parthenon']",
 'samaria': '[\'santiago\', \'salem\', \'springfield\', \'portsmouth\', \'aberdeen\', \'columbus\', \'victoria\', \'jackson\', \'hamilton\', \'cordoba\', \'cordova\', \'cartagena\', \'leon\', \'brunswick\', \'frankfort\', \'thebes\', \'syracuse\', \'manchester\', \'birmingham\', \'cambridge\', \'worcester\', \'newport\', \'memphis\', \'hyderabad\', \'utica\', \'tripoli\', \'odessa\', \'toledo\', \'valencia\', \'augusta\', \'portland\', \'greenville\', \'charleston\', \'kingston\', \'athens\', \'columbia\', \'florence\', \'vancouver\', \'oxford\', \'gloucester\', \'alexandria\', \'wilmington\', \'decatur\', \'lexington\', \'rochester\', \'a

In [6]:
!cd ../script 

In [19]:
from datetime import datetime
now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

!python add_path_score_for_cohyponyms_new.py True 0 100

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 15:34:24
reading cohyponyms: ../log/word_to_cohyponyms.txt
all instances: 12826
#instances chunk: 100
0
^C
Traceback (most recent call last):
  File "add_path_score_for_cohyponyms_new.py", line 88, in <module>
    word_to_cohyponyms_score = get_path_score_for_cohyponyms(word_to_cohyponyms, debug=debug)
  File "add_path_score_for_cohyponyms_new.py", line 54, in get_path_score_for_cohyponyms
    score = get_shortest_path_score(k, v, top_k_senses=2)
  File "add_path_score_for_cohyponyms_new.py", line 29, in get_shortest_path_score
    synsets2 = ewn.synsets(word2, pos='n')
  File "/home/chunhua/.local/lib/python3.8/site-packages/wn/_core.py", line 1205, in synsets
    return _find_helper(self, Synset, find_synsets, form, pos, ili=ili)
  File "/home/chunhua/.local/lib/python3.8/site-packages/wn/_core.py", line 1335, in _find_helper
    results = [
  File "/home/chunhua/.local/lib/python3.8/site-packages/wn/_core.py", line 1338, in <listcomp>
    for data in query_func(forms=

In [27]:
import wn
from wn.similarity import path
ewn = wn.Wordnet('ewn:2020')
paths = []
for synset1 in ewn.synsets('piano', 'n')[:2]:
    for synset2 in ewn.synsets('upright', 'n')[:2]:
        score = path(synset1, synset2)
        paths.append(score)
        print(f"{score} {synset1.lemmas()[0]} {synset2.lemmas()[0]}")
print(paths)

0.14285714285714285 forte-piano vertical
0.5 forte-piano upright
0.0625 piano vertical
0.058823529411764705 piano upright
[0.14285714285714285, 0.5, 0.0625, 0.058823529411764705]


In [20]:
import json 

json_path = '../log/word_to_cohyponyms_score_pl.json'
word_to_cohyponyms = json.load(open(json_path, 'r'))
word_to_cohyponyms['piano']

{'pianoforte': 1.0,
 'grand': 0.5,
 'pianola': 0.5,
 'upright': 0.5,
 'spinets': 0.3333,
 'gong': 0.3333,
 'clavichord': 0.3333,
 'klavier': 0.3333,
 'clavier': 0.3333,
 'dulcimers': 0.3333,
 'cithers': 0.3333,
 'accordions': 0.3333,
 'synthesiser': 0.3333,
 'synthesizers': 0.3333,
 'bones': 0.3333,
 'clappers': 0.3333,
 'castanets': 0.3333,
 'chimes': 0.3333,
 'cymbal': 0.3333,
 'tympan': 0.3333,
 'drum': 0.3333,
 'membranophones': 0.3333,
 'glockenspiel': 0.3333,
 'kettledrum': 0.3333,
 'timpani': 0.3333,
 'tympani': 0.3333,
 'lagerphone': 0.3333,
 'maraca': 0.3333,
 'xylophone': 0.3333,
 'marimba': 0.3333,
 'vibraharps': 0.3333,
 'vibes': 0.3333,
 'vibraphone': 0.3333,
 'banjos': 0.3333,
 'strings': 0.3333,
 'chordophones': 0.3333,
 'guitars': 0.3333,
 'kotos': 0.3333,
 'psalteries': 0.3333,
 'shamisens': 0.3333,
 'samisen': 0.3333,
 'sitars': 0.3333,
 'zithern': 0.3333,
 'zither': 0.3333,
 'celesta': 0.3333,
 'harpsichord': 0.25,
 'cembalos': 0.25,
 'contrabasses': 0.25,
 'harps': 

In [44]:
import pandas as pd
# path="../../log/bert-large-uncased/ALL/SWOWStrength/df_all_use_global_dap_True_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_anchor_source_WordNet_wns_False_wnf_False_swow_score_source_None.ALLSWOW.csv"
path = "../../log/bert-large-uncased/clsb/singular/df_all_use_global_dap_True_max_anchor_num_5_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_anchor_source_WordNet_wns_False_wnf_False_swow_score_source_None.CLSB.csv"
df = pd.read_csv(path)


In [43]:
df['obj_label']

0        vehicle
1         animal
2       predator
3        vehicle
4            van
          ...   
1018        food
1019       snack
1020      liquid
1021      animal
1022      mammal
Name: obj_label, Length: 1023, dtype: object

In [48]:
pd.options.display.max_colwidth = 1000
pd.options.display.max_columns = 1000
from collections import Counter, defaultdict 
print(f"#All instances: {len(df.index)}")
print(df.columns)
print(len(df.query('p10_def_dap==0').index))

df['missing_anchor'] = df['subj_anchors_wordnet_sg'].apply(lambda x:  1 if eval(x)[0] == 'MISSING' else 0)
display ( df['missing_anchor'].value_counts())

print ( Counter(df.query('p10_def_dap==0')['obj_label'].to_list()).most_common() )
df.query(f"p10_def_dap==0")[['sub_label_sg', 'obj_label', 'subj_anchors_wordnet_sg','obj_mask_def_dap']].sample(30)


#All instances: 1023
Index(['Unnamed: 0', 'sub_label', 'obj_label', 'masked_sentences', 'relation',
       'weight', 'feature_type', 'obj_label_single', 'sub_sister', 'uuid',
       'sub_position', 'sub_label_sg', 'sub_label_pl', 'def_sap', 'def_dap',
       'lsp_sap', 'lsp_dap', 'anchor_lsp_sap', 'sub_label_sgpl', 'obj_in_BERT',
       'obj_mask_sentence', 'obj_mask_sentence_score', 'obj_mask_def_sap',
       'obj_mask_def_sap_score', 'obj_mask_lsp_sap', 'obj_mask_lsp_sap_score',
       'subj_anchors_wordnet', 'subj_anchors_wordnet_sg',
       'subj_anchors_wordnet_pl', 'subj_anchors_wordnet_all',
       'subj_anchors_combined', 'anchor_num', 'def_dap_with_subj_anchor',
       'lsp_dap_with_subj_anchor', 'obj_mask_def_dap',
       'obj_mask_def_dap_score', 'obj_mask_lsp_dap', 'obj_mask_lsp_dap_score',
       'obj_label_sg', 'p1_sentence', 'p5_sentence', 'p10_sentence',
       'p1_def_sap', 'p5_def_sap', 'p10_def_sap', 'p1_lsp_sap', 'p5_lsp_sap',
       'p10_lsp_sap', 'p1_def_dap', 'p5

0    903
1    120
Name: missing_anchor, dtype: int64

[("['animal']", 33), ("['pet']", 16), ("['predator']", 14), ("['pest']", 13), ("['tool']", 13), ("['toy']", 11), ("['symbol']", 10), ("['food']", 10), ("['plant']", 9), ("['accessory']", 9), ("['liquid']", 9), ("['gift']", 7), ("['snack']", 6), ("['ingredient']", 5), ("['home']", 4), ("['vegetable']", 3), ("['treat']", 3), ("['spread']", 3), ("['baby']", 3), ("['cylinder']", 3), ("['disc']", 3), ("['bulb']", 3), ("['weed']", 3), ("['resource']", 3), ("['mammal']", 3), ("['shape']", 3), ("['achievement']", 2), ("['bowl']", 2), ("['container']", 2), ("['seed']", 2), ("['pulse']", 2), ("['car']", 2), ("['spirit']", 2), ("['material']", 2), ("['box']", 2), ("['antique']", 2), ("['weapon']", 2), ("['stick']", 2), ("['fruit']", 2), ("['device']", 2), ("['feature']", 2), ("['tube']", 2), ("['lever']", 2), ("['grape']", 2), ("['vehicle']", 2), ("['digit']", 2), ("['thistle']", 1), ("['award']", 1), ("['sink']", 1), ("['bunk']", 1), ("['queen']", 1), ("['warning']", 1), ("['signal']", 1), ("['b

,sub_label_sg,obj_label,subj_anchors_wordnet_sg,obj_mask_def_dap
1002,whisky,['liquid'],"['whiskey', 'irish', 'scotch', 'liquor', 'spirit']","['drink', 'liquor', 'whiskey', 'beer', 'alcohol', 'beverage', 'scotch', 'spirit', 'wine', 'gin']"
150,caravan,['home'],"['camper', 'van']","['vehicle', 'car', 'transport', 'automobile', 'bus', 'truck', 'wagon', 'transportation', 'carriage', 'trailer']"
231,colander,['bowl'],"['cullender', 'strainer', 'sieve', 'riddle', 'sifter']","['machine', 'puzzle', 'spoon', 'funnel', 'mixer', 'refrigerator', 'lever', 'device', 'knife', 'filter']"
376,gerbil,['pet'],"['gerbille', 'jird', 'rodent', 'gnawer', 'hedgehog']","['dog', 'mammal', 'animal', 'rabbit', 'weasel', 'pig', 'squirrel', 'bird', 'rat', 'cat']"
27,aubergine,['plant'],['MISSING'],"['diamond', 'stone', 'flower', 'rock', 'horn', 'crystal', 'grape', 'pearl', 'star', 'mineral']"
301,doughnut,['ring'],"['sinker', 'donut', 'paczki', 'friedcake', 'doughboy']","['pastry', 'dessert', 'candy', 'dough', 'bread', 'cookie', 'snack', 'cake', 'food', 'cereal']"
821,shawl,['accessory'],"['tallith', 'tallis', 'serape', 'sarape', 'cloak']","['scarf', 'dress', 'garment', 'fabric', 'skirt', 'textile', 'blanket', 'cloak', 'cloth', 'clothing']"
641,pansy,['flower'],['MISSING'],"['bird', 'pig', 'fish', 'consonant', 'person', 'candy', 'fruit', 'insect', 'pigeon', 'butterfly']"
380,gherkin,['fruit'],"['pickle', 'cucumber', 'cuke', 'caper', 'chowchow']","['fish', 'vegetable', 'crab', 'pastry', 'sausage', 'potato', 'candy', 'bread', 'sandwich', 'costume']"
498,kite,['toy'],"['check', 'cheque', 'giro', 'paycheck', 'overdraft']","['currency', 'aircraft', 'money', 'payment', 'balloon', 'sail', 'coin', 'airplane', 'bird', 'plane']"
